<a href="https://colab.research.google.com/github/21cs076/Real-Time-Detection-of-Landslides-and-Vehicle-Identification/blob/main/V_aug.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import albumentations as A
import cv2
import os
import numpy as np

# Define augmentation pipeline
transform = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.1),
    A.Rotate(limit=90, p=0.5),
    A.RandomBrightnessContrast(p=0.2),
], bbox_params=A.BboxParams(format='yolo', label_fields=['class_labels'], min_visibility=0.2))

def read_label(label_path):
    labels = []
    with open(label_path, 'r') as file:
        for line in file.readlines():
            if line.strip():
                class_id, x, y, w, h = map(float, line.strip().split())
                labels.append([class_id, x, y, w, h])
    return np.array(labels)

def save_label(label_path, labels):
    with open(label_path, 'w') as file:
        for label in labels:
            file.write(' '.join(map(str, label)) + '\n')

def augment_and_save(image_path, label_path, save_image_path, save_label_path, transform, num_augmented):
    image = cv2.imread(image_path)
    labels = read_label(label_path)

    if labels.size == 0:  # Skip augmentation if no labels are found
        return

    class_labels = labels[:, 0]
    bboxes = labels[:, 1:]

    for i in range(num_augmented):
        transformed = transform(image=image, bboxes=bboxes, class_labels=class_labels)
        augmented_image = transformed['image']
        augmented_bboxes = transformed['bboxes']

        if len(augmented_bboxes) == 0:  # Skip if no bounding boxes remain after augmentation
            continue

        augmented_labels = np.hstack((np.array(transformed['class_labels'])[:, np.newaxis], augmented_bboxes))

        augmented_image_path = os.path.join(save_image_path, f"aug_{i}_{os.path.basename(image_path)}")
        augmented_label_path = os.path.join(save_label_path, f"aug_{i}_{os.path.basename(label_path)}")

        cv2.imwrite(augmented_image_path, augmented_image)
        save_label(augmented_label_path, augmented_labels)

# Paths to your dataset images and labels
dataset_image_path = "/content/drive/MyDrive/Vehicle_Dataset/images"
dataset_label_path = "/content/drive/MyDrive/Vehicle_Dataset/labels"
augmented_image_path = "/content/drive/MyDrive/Vehicle_Dataset/aug/images"
augmented_label_path = "/content/drive/MyDrive/Vehicle_Dataset/aug/labels"

os.makedirs(augmented_image_path, exist_ok=True)
os.makedirs(augmented_label_path, exist_ok=True)

# Augment each image and its corresponding label in the dataset
for image_name in os.listdir(dataset_image_path):
    image_path = os.path.join(dataset_image_path, image_name)
    label_path = os.path.join(dataset_label_path, image_name.replace('.jpg', '.txt'))
    augment_and_save(image_path, label_path, augmented_image_path, augmented_label_path, transform, num_augmented=2)

In [ ]:
import shutil
import os

# Ensure the augmented directories exist
os.makedirs(augmented_image_path, exist_ok=True)
os.makedirs(augmented_label_path, exist_ok=True)

# Copy original images to the augmented images directory
for image_name in os.listdir(dataset_image_path):
    src_image_path = os.path.join(dataset_image_path, image_name)
    dst_image_path = os.path.join(augmented_image_path, image_name)
    shutil.copy(src_image_path, dst_image_path)

# Copy original labels to the augmented labels directory
for label_name in os.listdir(dataset_label_path):
    src_label_path = os.path.join(dataset_label_path, label_name)
    dst_label_path = os.path.join(augmented_label_path, label_name)
    shutil.copy(src_label_path, dst_label_path)

In [ ]:
import os
import shutil
import random

def main():
    # Define source paths
    dataset_image_path = "/content/drive/MyDrive/Vehicle_Dataset/nor/images"
    dataset_label_path = "/content/drive/MyDrive/Vehicle_Dataset/nor/labels"

    # Define destination paths for images and labels
    train_image_path = "/content/drive/MyDrive/Vehicle_Dataset/train/images/"
    test_image_path = "/content/drive/MyDrive/Vehicle_Dataset/test/images/"
    val_image_path = "/content/drive/MyDrive/Vehicle_Dataset/valid/images/"

    train_label_path = "/content/drive/MyDrive/Vehicle_Dataset/train/labels/"
    test_label_path = "/content/drive/MyDrive/Vehicle_Dataset/test/labels/"
    val_label_path = "/content/drive/MyDrive/Vehicle_Dataset/valid/labels/"

    # Create destination directories for images and labels
    os.makedirs(train_image_path, exist_ok=True)
    os.makedirs(test_image_path, exist_ok=True)
    os.makedirs(val_image_path, exist_ok=True)
    os.makedirs(train_label_path, exist_ok=True)
    os.makedirs(test_label_path, exist_ok=True)
    os.makedirs(val_label_path, exist_ok=True)

    # Get all image files (supports .jpg, .jpeg, .png)
    images = [f for f in os.listdir(dataset_image_path) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
    random.shuffle(images)

    total_images = len(images)
    train_split = int(0.8 * total_images)
    test_split = int(0.9 * total_images)  # 80% train, next 10% test, remaining 10% validation

    train_images = images[:train_split]
    test_images = images[train_split:test_split]
    val_images = images[test_split:]

    # Move image and corresponding label files for training dataset
    for image_file in train_images:
        # Move image
        src_image = os.path.join(dataset_image_path, image_file)
        dst_image = os.path.join(train_image_path, image_file)
        shutil.move(src_image, dst_image)

        # Build corresponding label file name (assumed to be .txt)
        label_file = os.path.splitext(image_file)[0] + ".txt"
        src_label = os.path.join(dataset_label_path, label_file)
        dst_label = os.path.join(train_label_path, label_file)
        if os.path.exists(src_label):
            shutil.move(src_label, dst_label)
        else:
            print(f"Warning: No label file found for training image: {image_file}")

    # Move image and corresponding label files for testing dataset
    for image_file in test_images:
        src_image = os.path.join(dataset_image_path, image_file)
        dst_image = os.path.join(test_image_path, image_file)
        shutil.move(src_image, dst_image)

        label_file = os.path.splitext(image_file)[0] + ".txt"
        src_label = os.path.join(dataset_label_path, label_file)
        dst_label = os.path.join(test_label_path, label_file)
        if os.path.exists(src_label):
            shutil.move(src_label, dst_label)
        else:
            print(f"Warning: No label file found for testing image: {image_file}")

    # Move image and corresponding label files for validation dataset
    for image_file in val_images:
        src_image = os.path.join(dataset_image_path, image_file)
        dst_image = os.path.join(val_image_path, image_file)
        shutil.move(src_image, dst_image)

        label_file = os.path.splitext(image_file)[0] + ".txt"
        src_label = os.path.join(dataset_label_path, label_file)
        dst_label = os.path.join(val_label_path, label_file)
        if os.path.exists(src_label):
            shutil.move(src_label, dst_label)
        else:
            print(f"Warning: No label file found for validation image: {image_file}")

    print(f"Moved {len(train_images)} images and their corresponding labels to training set.")
    print(f"Moved {len(test_images)} images and their corresponding labels to testing set.")
    print(f"Moved {len(val_images)} images and their corresponding labels to validation set.")

if __name__ == "__main__":
    main()

In [ ]:
import yaml
data = {
    'train': "/content/drive/MyDrive/Vehicle_Dataset/train/images",
    'val': "/content/drive/MyDrive/Vehicle_Dataset/valid/images",
    'test': "/content/drive/MyDrive/Vehicle_Dataset/test/images",
    'nc': 5,
    'names': ['camping car', 'car', 'pickup', 'truck', 'van']
}

with open("/content/drive/MyDrive/Vehicle_Dataset/data.yaml", 'w') as file:
    yaml.dump(data, file, default_flow_style=False)